### Hyperparams

In [47]:
import torch

batch_size = 32
block_size = 8
max_iters = 5_000
eval_interval = 300
learning_rate = 1e-3
eval_iters = 200
train_split = 0.9
device = 'cuda'

n_embd = 32

### Data

In [48]:
with open ('/data/tinyshakespeare.txt', 'r', encoding='utf-8') as file:
    text = file.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s] # encoder: map string to char ids
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: map integer ids to a string

data = torch.tensor(encode(text), dtype=torch.long)

# Splitting the dataset into training and validation sets
n = int(train_split * len(data)) # 90% training, 10% validation
train_data, validation_data = data[:n], data[n:]

def get_batch(split):
  # batch of data of inputs x and targets y
  data = train_data if split == 'train' else validation_data
  # array(batch_size) of random offsets within data
  ix = torch.randint(len(data) - block_size, (batch_size,))

  # Context and target sequences
  # ? Why a single sequence is not used?
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  x, y = x.to(device), y.to(device)

  return x, y

### Loss

In [49]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()

  for split in ['train', 'validation']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()

    out[split] = losses.mean()

  model.train()
  return out

### Model

In [55]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)

class Head(nn.Module):
  """ One self-attention head"""
  def __init__(self, head_size):
    super().__init__()

    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)

    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, x):
    B, T, C = x.shape

    k = self.key(x)
    q = self.query(x)

    # **0.5 is to avoid softmax from getting too peaky
    # when incoming values are large/spread out.
    wei = q @ k.transpose(-2, -1) * C**-0.5
    wei = wei.masked_fill(
      self.tril[:T, :T] == 0,
      float('-inf')
    )
    wei = F.softmax(wei, dim = -1)

    v = self.value(x)
    out = wei @ v

    return out

class MultiHeadAttention(nn.Module):
  """ Multiple heads of self-attention in parallel """

  def __init__(self, num_heads, head_size):
    super().__init__()

    self.heads = nn.ModuleList(
      [Head(head_size) for _ in range(num_heads)]
    )

  def forward(self, x):
    return torch.cat([h(x) for h in self.heads], dim=-1)


class FeedForward(nn.Module):
  """ Linear layer with ReLU activation in between """

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(n_embd, n_embd),
      nn.ReLU(),
    )

  def forward(self, x):
    return self.net(x)


class Block(nn.Module):
  """ Transformer block: communication (attention) followed by computation (feedforward) """

  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)

  def forward(self, x):
    x = self.sa(x)
    x = self.ffwd(x)
    return x


class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()

    # Video says: each token directly reads off the logits for the next token
    # from the lookup table.
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)

    self.sa_head = MultiHeadAttention(4, n_embd // 4)
    self.ffwd = FeedForward(n_embd)

    # self.sa_head = Head(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    # idx and targets are (B, T) tensors of integers, Batch, Time
    # Token embedding table will return shape of (Batch, Time, Channel)
    # Batch - number of sequences
    # Time - number of tokens in each sequence
    # Channel - Embeddings vector for a token?
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))

    x = tok_emb + pos_emb
    x = self.sa_head(x)      # (B, T, C), one self-attention head
    x = self.ffwd(x)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      return logits, None

    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)

    # Cross entropy expects input to be in a 2D tensor of shape (Batch*Time, Channel)
    loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is a (Batch, Time) tensor of integers, representing current context
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]

      # compute the predictions
      logits, loss = self(idx_cond) # (B, T, C)

      # -1 makes very little sense for a bigram model,
      # as we're essentially throwing away everything except the very last token in a batch
      # to make our prediction.
      # This is done in such a way only to allow for easier transition to an N-gram model later.
      logits = logits[:, -1, :] # (B, C)

      # Probabilities from logits
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

      # append to the currently running context
      idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
    return idx

model = BigramLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

### Train

In [56]:
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f'Iter {iter}, Train loss: {losses["train"]}, Validation loss: {losses["validation"]}')

  # Sample batch of data
  xb, yb = get_batch('train')

  # Eval
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

Iter 0, Train loss: 4.164020538330078, Validation loss: 4.162225246429443
Iter 300, Train loss: 2.755855083465576, Validation loss: 2.7701339721679688
Iter 600, Train loss: 2.5916225910186768, Validation loss: 2.5889241695404053
Iter 900, Train loss: 2.5146775245666504, Validation loss: 2.5048437118530273
Iter 1200, Train loss: 2.467432975769043, Validation loss: 2.4645252227783203
Iter 1500, Train loss: 2.4242730140686035, Validation loss: 2.448246479034424
Iter 1800, Train loss: 2.404893398284912, Validation loss: 2.4144601821899414
Iter 2100, Train loss: 2.3868961334228516, Validation loss: 2.387389898300171
Iter 2400, Train loss: 2.360210418701172, Validation loss: 2.3794562816619873
Iter 2700, Train loss: 2.350212812423706, Validation loss: 2.355624198913574
Iter 3000, Train loss: 2.3277995586395264, Validation loss: 2.34500789642334
Iter 3300, Train loss: 2.3108701705932617, Validation loss: 2.3343567848205566
Iter 3600, Train loss: 2.3040761947631836, Validation loss: 2.30837273

### Generate

In [52]:
ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
batch = model.generate(ctx, max_new_tokens=500)
logits = batch[0].tolist()

print(decode(logits))



ASTIf ored visy ubd I theolugh mead
canisysunty do! O,
St
Yoursto;
As
The asard.

RUCKE:
Sout, hout shein's trepueche bode apteres,
Or was brit, andsectle: splut, hereart leif me, ich ade's may!

TASARD:
Feads, may bed I'll him worl I:
Wely rieken your oneters wore foriock in I thise omplacts thou fad,
Ays, thouch reay, inu, eromerive.
I'd you rive, cound nowere cays.
I suppet, weld
hat agot th, woulde, with mier, plle pontlance kentat be't;
And I gring mist quough oaken-s's ball the of it come
